In [355]:
import re
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize, wordpunct_tokenize, sent_tokenize, RegexpTokenizer, regexp_tokenize, WhitespaceTokenizer
from nltk.tokenize import MWETokenizer
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords, wordnet
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk
from nltk.book import *
import spacy
import string
from collections import Counter

from nlp_cleaning import *

pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', -1)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [159]:
df = pd.read_csv("attraction_point_reviews.csv")
# df.tail()

# Cleaning the Text 

As part of cleaning the text before tokenizing it, various steps will be undertaken. 

#### But first, to get an initial feel for the data, the following steps are being undertaken:
1. All reviews are combined into a giant single string corpus.
1. All words are changed to lower letter case
1. website links & email ids are dropped
1. A lot of words are wrongly connected with punctuations. Simply dropping the punctuations will connect these words. These will be substituted with whitespace for the following punctuations: <.*?>;-!()/,:&—\ 
1. Everything except for letters & whitespace is dropped with no substitutions in between.
1. The string is tokenized on whitespace.
1. It is then converted into a word counter using FreqDist to explore frequencies and look at the various words used.

In [312]:
all_reviews = ' '.join(df.review_text.tolist())
all_reviews = all_reviews.lower()
all_reviews = re.sub('http\S+', '' , all_reviews)
all_reviews = re.sub('\S*@\S+', '', all_reviews)

all_reviews = re.sub(r'[<.*?>;\-!()/,:&—\\]+', ' ', all_reviews)
all_reviews = re.sub(r'[^A-Za-z\s]', '', all_reviews)
words = WhitespaceTokenizer().tokenize(all_reviews)
words_count = FreqDist(words)

long_words = [w for w in words_count if len(w) > 15]
small_words = [w for w in words_count if len(w) < 4]

**The above process was carried out iteratively to ensure as many long words as possible can be captured properly and not words arbitrarily combined together using punctuations**

## COME BACK TO TRY MORE CLEANING FOR:
1. letter repeats (tweet tokenizer)
1. Spell Check
1. Different languages
1. Named Entity Extraction
3. Identifying actual hypentated words instead of separating them 
1. Removing words that are less than 4 letters perhaps - but selectively

**The cleaning strategy was applied to the reviews in the dataframe.**

In [348]:
df['review_clean'] = df.review_text.map(cleaning)
df.review_clean.sample()

6958    teens left mom and dad in the dust    we took frequent stops due to the steep paved walkway   take plenty of water     best to visit early in the morning to avoid the crowds   be aware not to venture past the railings  warning signs  etc    this area has had four deaths this summer   stay out of the water period   stay off the rocks period   one  yr old teen died on aug   due to a fall on the rocks take no strollers or wheelchairs    not a venture for those in a wheelchair   travel light and wear proper walking shoes   do not wear your flip flops or sandals 
Name: review_clean, dtype: object

The words are then lemmatized after cleaning, to reduce the variety of words itself in the corpus. Additionally, all pronouns are dropped.

In [367]:
%%time
df['review_lemma'] = df.review_clean.map(spacy_lemmatization)

Wall time: 2min 24s


In [369]:
df[['review_lemma','review_clean']].sample(2)

,review_lemma,review_clean
2681,the park be huge realize that can not be experience in just a couple of day the pass be magnificent husband love to do the loop from to tioga pass and back around to twain harte on motorcycle take about hour,the park is huge you realize that it cant be experience in just a couple of days the pass is magnificent my husband loves to do the loop from to tioga pass and back around to twain harte on his motorcycle takes him about hours
2391,with lake and meadow everywhere this road be the drive in heaven take from lee vine to the olmstead point wish to go further the next time,with lakes and meadows everywhere this road is the drive in heaven we took it from lee vining to the olmstead point we wish to go further the next time


Final level of filtering 

In [370]:
stopwords_list = stopwords.words('english')
stopwords_list

words_count = FreqDist(df.review_lemma[1])

long_words = [w for w in words_count if len(w) > 15]
small_words = [w for w in words_count if len(w) < 4]

[&#39;i&#39;,
 &#39;me&#39;,
 &#39;my&#39;,
 &#39;myself&#39;,
 &#39;we&#39;,
 &#39;our&#39;,
 &#39;ours&#39;,
 &#39;ourselves&#39;,
 &#39;you&#39;,
 &quot;you&#39;re&quot;,
 &quot;you&#39;ve&quot;,
 &quot;you&#39;ll&quot;,
 &quot;you&#39;d&quot;,
 &#39;your&#39;,
 &#39;yours&#39;,
 &#39;yourself&#39;,
 &#39;yourselves&#39;,
 &#39;he&#39;,
 &#39;him&#39;,
 &#39;his&#39;,
 &#39;himself&#39;,
 &#39;she&#39;,
 &quot;she&#39;s&quot;,
 &#39;her&#39;,
 &#39;hers&#39;,
 &#39;herself&#39;,
 &#39;it&#39;,
 &quot;it&#39;s&quot;,
 &#39;its&#39;,
 &#39;itself&#39;,
 &#39;they&#39;,
 &#39;them&#39;,
 &#39;their&#39;,
 &#39;theirs&#39;,
 &#39;themselves&#39;,
 &#39;what&#39;,
 &#39;which&#39;,
 &#39;who&#39;,
 &#39;whom&#39;,
 &#39;this&#39;,
 &#39;that&#39;,
 &quot;that&#39;ll&quot;,
 &#39;these&#39;,
 &#39;those&#39;,
 &#39;am&#39;,
 &#39;is&#39;,
 &#39;are&#39;,
 &#39;was&#39;,
 &#39;were&#39;,
 &#39;be&#39;,
 &#39;been&#39;,
 &#39;being&#39;,
 &#39;have&#39;,
 &#39;has&#39;,
 &#39;had&#39;,
 &#3

In [372]:
df.to_csv("Reviews_cleaned_for_NLP.csv", index = False)

### Tokenization 

In [138]:
from nltk.tokenize import word_tokenize

my_text = "Hi Mr. Smith! I’m going to buy some vegetables (tomatoes and cucumbers) from \
the store. Should I pick up some black-eyed peas as well?"

print(word_tokenize(my_text))

# (N-Grams)

from nltk.util import ngrams
my_words = word_tokenize(my_text) # This is the list of all words
twograms = list(ngrams(my_words,2)) # This is for two-word combos, but can pick any n
print(twograms)

# Regular Expressions

from nltk.tokenize import RegexpTokenizer

# RegexpTokenizer with whitespace delimiter
whitespace_tokenizer = RegexpTokenizer("\s+", gaps=True)
print(whitespace_tokenizer.tokenize(my_text))

# RegexpTokenizer to match only capitalized words
cap_tokenizer = RegexpTokenizer("[A-Z]['\w]+")
print(cap_tokenizer.tokenize(my_text))

from nltk.tokenize import regexp_tokenize, wordpunct_tokenize, blankline_tokenize

s = "Good muffins cost $3.88\nin New York.  Please buy me\ntwo of them.\n\nThanks."
regexp_tokenize(s, pattern='\w+|\$[\d\.]+|\S+')

wordpunct_tokenize(s)

blankline_tokenize(s)


[&#39;Hi&#39;, &#39;Mr.&#39;, &#39;Smith&#39;, &#39;!&#39;, &#39;I&#39;, &#39;’&#39;, &#39;m&#39;, &#39;going&#39;, &#39;to&#39;, &#39;buy&#39;, &#39;some&#39;, &#39;vegetables&#39;, &#39;(&#39;, &#39;tomatoes&#39;, &#39;and&#39;, &#39;cucumbers&#39;, &#39;)&#39;, &#39;from&#39;, &#39;the&#39;, &#39;store&#39;, &#39;.&#39;, &#39;Should&#39;, &#39;I&#39;, &#39;pick&#39;, &#39;up&#39;, &#39;some&#39;, &#39;black-eyed&#39;, &#39;peas&#39;, &#39;as&#39;, &#39;well&#39;, &#39;?&#39;]
[(&#39;Hi&#39;, &#39;Mr.&#39;), (&#39;Mr.&#39;, &#39;Smith&#39;), (&#39;Smith&#39;, &#39;!&#39;), (&#39;!&#39;, &#39;I&#39;), (&#39;I&#39;, &#39;’&#39;), (&#39;’&#39;, &#39;m&#39;), (&#39;m&#39;, &#39;going&#39;), (&#39;going&#39;, &#39;to&#39;), (&#39;to&#39;, &#39;buy&#39;), (&#39;buy&#39;, &#39;some&#39;), (&#39;some&#39;, &#39;vegetables&#39;), (&#39;vegetables&#39;, &#39;(&#39;), (&#39;(&#39;, &#39;tomatoes&#39;), (&#39;tomatoes&#39;, &#39;and&#39;), (&#39;and&#39;, &#39;cucumbers&#39;), (&#39;cucumbers&#3

### Preprocessing: Stop Words

In [137]:
from nltk.corpus import stopwords
set(stopwords.words('english'))

#Example impact with code

my_text = ["Hi Mr. Smith! I’m going to buy some vegetables (tomatoes and cucumbers) from \
the store. Should I pick up some black-eyed peas as well?"]

# Incorporate stop words when creating the count vectorizer
cv = CountVectorizer(stop_words='english')
X = cv.fit_transform(my_text)
pd.DataFrame(X.toarray(), columns=cv.get_feature_names())

### POS Tagging With NLTK

In [128]:
from nltk.tag import pos_tag
my_text = "James Smith lives in the United States."
tokens = pos_tag(word_tokenize(my_text))
print(tokens)

#For help on the codes, use the below
# nltk.help.upenn_tagset()

[(&#39;James&#39;, &#39;NNP&#39;), (&#39;Smith&#39;, &#39;NNP&#39;), (&#39;lives&#39;, &#39;VBZ&#39;), (&#39;in&#39;, &#39;IN&#39;), (&#39;the&#39;, &#39;DT&#39;), (&#39;United&#39;, &#39;NNP&#39;), (&#39;States&#39;, &#39;NNPS&#39;), (&#39;.&#39;, &#39;.&#39;)]


### Named Entity Recognition

In [132]:
from nltk.chunk import ne_chunk
my_text = "James Smith lives in the United States."
tokens = pos_tag(word_tokenize(my_text)) # this labels each word as a part of speech
entities = ne_chunk(tokens) # this extracts entities from the list of words
# help(entities)

### Compound Term Extraction

In [ ]:
from nltk.tokenize import MWETokenizer # multi-word expression
my_text = "You all are the greatest students of all time."
mwe_tokenizer = MWETokenizer([('You','all'), ('of', 'all', 'time')])
mwe_tokens = mwe_tokenizer.tokenize(word_tokenize(my_text))
mwe_tokens